# 平均報酬におけるUCRL2アルゴリズム

参考：
* [REGAL: A Regularization based Algorithm for Reinforcement Learning in Weakly Communicating MDPs](https://arxiv.org/abs/1205.2661)

Average rewardでSpanを外すアルゴリズムについて見ていきます

表記：

* $T$ステップまでに得られた期待報酬：$R^{\mathcal{G}}\left(s_1, T\right)=\mathbb{E}\left[\sum_{t=1}^T r_t\right]$
* リグレット：$\Delta^{\mathcal{G}}\left(s_1, T\right)=\lambda^{\star} T-R^{\mathcal{G}}\left(s_1, T\right)$
    * $\lambda^\star$は最適な報酬．
* Ergodic：全ての方策について，マルコフ連鎖が単一のRecurrent classになる（つまり，任意の状態から任意の状態に到達可能）．
* Unichain：全ての方策について，マルコフ連鎖が単一のRecurrent classと，（空かもしれない）transientな状態集合になる．
* Communicating：任意の$s_1, s_2 \in \mathcal{S}$について，$s_1$から$s_2$に遷移可能な**方策が存在する**．
* Weakly communicating：$\mathcal{S}$が２つに分割可能
    1. Communicatingな状態集合
    2. 任意の方策に対してTransientな状態集合

ちなみに，こうした構造がない限り，リグレットの保証をつけることは不可能です．例えば，最適な報酬を得る状態に絶対にいけない状態にエージェントが配置されてしまった場合，無理です．
今回はWeakly communicatingを考えます．

* Weakly communicatingでは，最適ゲイン$\lambda^\star$は状態に非依存です．次の最適方程式を満たします：
$$
h^{\star}+\lambda^{\star} \mathbf{e}=\max _{a \in \mathcal{A}}\left(r(s, a)+P_{s, a}^{\top} h^{\star}\right) = \mathcal{T} h^{\star}
$$
ここで，$h^\star$はバイアスベクトルです．$h^\star + c\boldsymbol{e}$もバイアスベクトルになります．
* スパン：$\operatorname{sp}(h):=\max _{s \in \mathcal{S}} h(s)-\min _{s \in \mathcal{S}} h(s)$



## REGALアルゴリズム


今回見るREGALアルゴリズムは$\tilde{O}\left(\operatorname{sp}\left(h^{\star}(M)\right) S \sqrt{A T}\right)$を達成します．

---

**補足：半径について**

REGAL以前のアルゴリズムでは
$$
\tilde{O}(D(M) S \sqrt{A T})
$$
のリグレットが達成されてきました．ここで，$D(M)$はMDP$M$の直径です．
$$
D(M):=\max _{s_1 \neq s_2} \min _\pi T_{s_1 \rightarrow s_2}^\pi
$$
$T_{s_1 \rightarrow s_2}^\pi$は$\pi$が$s_1$から$s_2$へ到達する期待ステップ数を表します．

しばしば他の直径が使われることもあります：
$$
\begin{aligned}
D_{\text {worst }}(M) & :=\max _\pi \max _{s_1 \neq s_2} T_{s_1 \rightarrow s_2}^\pi \\
D_{\text {opt }}(M) & :=\min _{\pi \in \mathcal{O}} \max _{s_1 \neq s_2} T_{s_1 \rightarrow s_2}^\pi
\end{aligned}
$$
明らかに，$D \leq D_\text{opt} \leq D_\text{worst}$です．
今回は次の片道直径を使います：

$$
D_{\mathrm{ow}}(M):=\max _s \min _\pi T_{s_1 \rightarrow \bar{s}}^\pi
$$

ここで，$\bar{s}=\operatorname{argmax}_s h^{\star}(s)$です．[RL_AverageReward_lemma.ipynb](RL_useful_lemma/RL_AverageReward_lemma.ipynb)で示しますが，$\operatorname{sp}\left(h^{\star}\right) \leq D_{\text {ow }} \leq D$が成り立ちます．
実はこの片道直径を使うと，今回のバウンドがタイトであることが示せます（Theorem 6）．

---

REGALアルゴリズムは，MDPのConfidence setから
$$
\lambda^{\star}(M)-C_k \operatorname{sp}\left(h^{\star}(M)\right)
$$
が最大になるようなMDP$M$を見つけます．つまり，最適ゲインを大きく，そしてスパンを小さめに抑えるMDPを選択します．

各エピソード$k=1, 2, \dots$において，
1. $t_k$は現在の時刻です
2. $\mathcal{M}^k$を$\left\|P_{s, a}-\hat{P}_{s, a}^t\right\|_1 \leq \sqrt{\frac{12 S \log (2 A t / \delta)}{\max \{N(s, a, t), 1\}}}$を満たすconfidence setとします
3. $\lambda^{\star}(M)-C_k \operatorname{sp}\left(h^{\star}(M)\right)$を最大にするMDP$M^k$を選びます
4. $\pi^k$を$M^k$のゲイン最適方策とします
5. どれかの$s, a$が$N_k(s, a)$回訪問されるまで$\pi^k$を実行します．

$v_k(s, a)=N_{k+1}(s, a)-N_k(s, a)$を$k$エピソード目に$(s, a)$が訪問された回数とします．


### リグレット解析

* $\ell_k=\sum_{s, a} v_k(s, a)$：エピソード$k$の長さ
* $\lambda\left(M^k\right), h^{\star}\left(M^k\right)$を$\lambda_k$，$h_k^\star$とします．
* $\tilde{P}_k$は$M^k$での遷移確率．$P_k$は$M$での遷移確率とします．
* $h^\star$と$h^\star_k$はそれぞれ
$$
\operatorname{sp}\left(h^{\star}\right)=\left\|h^{\star}\right\|_{\infty} \text { and } \operatorname{sp}\left(h_k^{\star}\right)=\left\|h_k^{\star}\right\|_{\infty}
$$
が満たされるように正規化されているとします．
$$\begin{aligned} & \mathbf{v}_k(s)=v_k\left(s, \pi^k(s)\right) \\ & \mathbf{r}_k(s)=r\left(s, \pi^k(s)\right)\end{aligned}$$
なるベクトルを導入します．このとき，
$$
h_k^{\star}+\lambda_k^{\star} \mathbf{e}=\mathbf{r}_k+\tilde{\mathbf{P}}_k h_k^{\star}
$$
です．



---

**エピソード数のバウンド**

まず，次の定理が成立します：

$T \geq $SA$ステップ目までのエピソードの数$m$は,
$$
m \leq S A \log _2\left(\frac{8 T}{S A}\right)
$$
でバウンドされる．

証明は[RL_useful_lemma/RL_AverageReward_lemma.ipynb](RL_useful_lemma/RL_AverageReward_lemma.ipynb)参照．

---

**リグレットのバウンドの書き換え**

$$\Delta_k=\sum_{s, a} v_k(s, a)\left[\lambda^{\star}-r(s, a)\right]$$
を$k$エピソード内のリグレットとします．このとき，リグレットは
$$
\sum_{k \in G} \Delta_k+\sum_{k \in B} \Delta_k
$$
と同じです．ここで
$$
G=\left\{k: M \in \mathcal{M}^k\right\} \text { and } B=\{1, \ldots, m\}-G
$$
としました．つまり，$G$はGood event, $B$はBAD eventです．

このとき，次が成立します（つまりBAD eventはあまり起きません）：

確率 $1- \delta$以上で，
$$
\sum_{k \in R} \Delta_k \leq \sqrt{T}
$$

TODO:証明

---

